# 目标

1. 预测：把原图切成cut_size1大小的图片，需要补全，归一化后切成cut_size2大小的图片，需要补全
2. 后期处理：把预测出来的cut_size2大小的图片合成cut_size1大小，然后再合成原图大小

# 思路

1. 把原图切成cut_size1大小的图片，右边、下面和右下角可能有边角需要补全，归一化后切成cut_size2大小的图片，右边、下面和右下角可能有边角需要补全
2. 把预测出来的cut_size2大小的图片合成cut_size1大小，有多余的部分需要去掉，然后再合成原图大小

# 导包和读取原图

In [44]:
import cv2
import tifffile as tiff
import os
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# FILE_2015 = '../preliminary/quickbird2015_preliminary_2.tif'
# FILE_2017 = '../preliminary/quickbird2017_preliminary_2.tif'
# final阶段图片
FILE_2015 = '../preliminary/quarterfinals_2015.tif'
FILE_2017 = '../preliminary/quarterfinals_2017.tif'
img_2015 = tiff.imread(FILE_2015).transpose([1, 2, 0])
img_2017 = tiff.imread(FILE_2017).transpose([1, 2, 0])
print img_2015.shape

(4000, 15106, 4)


# 调用的归一化函数

In [45]:
def scale_percentile_n(matrix):
    w, h, d = matrix.shape
    matrix = np.reshape(matrix, [w * h, d]).astype(np.float64)
    # Get 2nd and 98th percentile
    mins = np.percentile(matrix, 1, axis=0)
#     print mins
    maxs = np.percentile(matrix, 99, axis=0) - mins
#     print maxs
    matrix = (matrix - mins[None, :]) / maxs[None, :]
    matrix = np.reshape(matrix, [w, h, d])
    matrix = matrix.clip(0, 1)
    return matrix

# 把原图切成960大小补全后归一化

In [46]:
# 设置通道数
channel = [0, 1, 2]
# channel = [0, 1, 2]
img_2015 = img_2015[:,:,channel]
img_2017 = img_2017[:,:,channel]
print img_2015.shape

(4000, 15106, 3)


In [48]:
IM_ROWS = img_2015.shape[0]
IM_COLS = img_2015.shape[1]
cut_size1 = 960

DATA_DIR = 'output'

if not os.path.exists(DATA_DIR + '/train960_normalize'):
    os.makedirs(DATA_DIR + '/train960_normalize')

years = [2015, 2017]

for year in years:
    img = eval('img_{}'.format(year))
    for i in xrange(int(IM_ROWS // cut_size1)):
        for j in xrange(int(IM_COLS // cut_size1)):
            ss = '{}/train960_normalize/{}_{}_{}_{}.jpg'.format(DATA_DIR, year, i, j, cut_size1)
            if os.path.exists(ss):
                continue
            tiff.imsave(ss, np.array(scale_percentile_n(img[i*cut_size1:(i+1)*cut_size1,j*cut_size1:(j+1)*cut_size1])*255, dtype=np.uint8))

    if IM_ROWS%cut_size1 != 0:
        i += 1
        for k in xrange(int(IM_COLS // cut_size1)):
            ss = '{}/train960_normalize/{}_{}_{}_{}.jpg'.format(DATA_DIR, year, i, k, cut_size1)
            if os.path.exists(ss):
                continue
            tiff.imsave(ss, np.array(scale_percentile_n(img[(IM_ROWS-cut_size1):IM_ROWS, k*cut_size1:(k+1)*cut_size1])*255, dtype=np.uint8))

    if IM_COLS%cut_size1 != 0:
        j += 1
        for k in xrange(int(IM_ROWS // cut_size1)):
            ss = '{}/train960_normalize/{}_{}_{}_{}.jpg'.format(DATA_DIR, year, k, j, cut_size1)
            if os.path.exists(ss):
                continue
            tiff.imsave(ss, np.array(scale_percentile_n(img[k*cut_size1:(k+1)*cut_size1, (IM_COLS-cut_size1):IM_COLS])*255, dtype=np.uint8))

    if IM_ROWS%cut_size1 != 0 and IM_COLS%cut_size1 != 0:
        ss = '{}/train960_normalize/{}_{}_{}_{}.jpg'.format(DATA_DIR, year, i, j, cut_size1)
        if os.path.exists(ss):
                continue
        tiff.imsave(ss, np.array(scale_percentile_n(img[(IM_ROWS-cut_size1):IM_ROWS, (IM_COLS-cut_size1):IM_COLS])*255, dtype=np.uint8))
        
print('Generation completed')

Generation completed


# 把cut_size1大小的图片切成cut_size2大小的图片，需要补全

In [49]:
cut_size2 = 160

if not os.path.exists(DATA_DIR + '/result_image'):
    os.makedirs(DATA_DIR + '/result_image')

for year in years:
    for i in xrange(int(IM_ROWS // cut_size1)+1):
        for j in xrange(int(IM_COLS // cut_size1)+1):
            ss = '{}/train960_normalize/{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, cut_size1)
            img = cv2.imread(ss, -1)
            for x in xrange(int(cut_size1 // cut_size2)):
                for y in xrange(int(cut_size1 // cut_size2)):
                    ss1 = '{}/result_image/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, y, cut_size2)
                    tiff.imsave(ss1, np.array(img[x*cut_size2:(x+1)*cut_size2, y*cut_size2:(y+1)*cut_size2], dtype=np.uint8))

            if cut_size1%cut_size2 != 0:
                x += 1
                y += 1
                for k in xrange(int(cut_size1 // cut_size2)):
                    ss1 = '{}/result_image/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, k, cut_size2)
                    tiff.imsave(ss1, np.array(img[(cut_size1-cut_size2):cut_size1, k*cut_size2:(k+1)*cut_size2], dtype=np.uint8))
                    ss2 = '{}/result_image/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, k, y, cut_size2)
                    tiff.imsave(ss2, np.array(img[k*cut_size2:(k+1)*cut_size2, (cut_size1-cut_size2):cut_size1], dtype=np.uint8))

                ss3 = '{}/result_image/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, y, cut_size2)
                tiff.imsave(ss3, np.array(img[(cut_size1-cut_size2):cut_size1, (cut_size1-cut_size2):cut_size1], dtype=np.uint8))

print('Generation completed')

TypeError: 'NoneType' object has no attribute '__getitem__'

# 生成对应的掩膜图片假数据

In [56]:
if not os.path.exists(DATA_DIR + '/result_pred'):
    os.makedirs(DATA_DIR + '/result_pred')
    
for year in years:
    for i in xrange(int(IM_ROWS // cut_size1)+1):
        for j in xrange(int(IM_COLS // cut_size1)+1):
            for x in xrange(int(cut_size1 // cut_size2)):
                for y in xrange(int(cut_size1 // cut_size2)):
                    ss1 = '{}/result_image/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, y, cut_size2)
                    ss = '{}/result_pred/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, y, cut_size2)
                    img = tiff.imread(ss1)
                    tiff.imsave(ss, np.array(img[:,:,0]>128, dtype=np.uint8))

            if cut_size1%cut_size2 != 0:
                x += 1
                y += 1
                for k in xrange(int(cut_size1 // cut_size2)):
                    ss1 = '{}/result_image/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, k, cut_size2)
                    ss = '{}/result_pred/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, y, cut_size2)
                    img = tiff.imread(ss1)
                    tiff.imsave(ss, np.array(img[:,:,0]>128, dtype=np.uint8))
                    ss2 = '{}/result_image/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, k, y, cut_size2)
                    ss = '{}/result_pred/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, y, cut_size2)
                    img = tiff.imread(ss2)
                    tiff.imsave(ss, np.array(img[:,:,0]>128, dtype=np.uint8))

                ss3 = '{}/result_image/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, y, cut_size2)
                ss = '{}/result_pred/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, y, cut_size2)
                img = tiff.imread(ss3)
                tiff.imsave(ss, np.array(img[:,:,0]>128, dtype=np.uint8))

print('Generation completed')

Generation completed


# 合并成cut_size1大小的图片

In [70]:
import cv2
import tifffile as tiff
import os
import matplotlib.pyplot as plt
import numpy as np

cut_size2 = 160

%matplotlib inline

DATA_DIR = 'output'
if not os.path.exists(DATA_DIR+'/result_pred960'):
    os.makedirs(DATA_DIR+'/result_pred960')

for year in years:
    img = eval('img_{}'.format(year))
    for i in xrange(int(IM_ROWS // cut_size1) +1 ):
        for j in xrange(int(IM_COLS // cut_size1) + 1):
            img_pred = np.ones((cut_size1, cut_size1))
            ss = '{}/result_pred960/{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, cut_size1)
            if os.path.exists(ss):
                continue
            for x in xrange(int(cut_size1 // cut_size2)):
                for y in xrange(int(cut_size1 // cut_size2)):
                    ss1 = '{}/result_pred/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, y, cut_size2)
                    img_pred[x*cut_size2:(x+1)*cut_size2, y*cut_size2:(y+1)*cut_size2] = tiff.imread(ss1)

            if cut_size1%cut_size2 != 0:
                x += 1
                y += 1
                for k in xrange(int(cut_size1 // cut_size2)):
                    ss1 = '{}/result_pred/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, k, cut_size2)
                    img_pred[(cut_size1-cut_size2):cut_size1, k*cut_size2:(k+1)*cut_size2] = tiff.imread(ss1)
                    ss2 = '{}/result_pred/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, k, y, cut_size2)
                    img_pred[k*cut_size2:(k+1)*cut_size2, (cut_size1-cut_size2):cut_size1] = tiff.imread(ss1)

                ss3 = '{}/result_pred/{}_{}_{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, x, y, cut_size2)
                img_pred[(cut_size1-cut_size2):cut_size1, (cut_size1-cut_size2):cut_size1] = tiff.imread(ss1)
            tiff.imsave(ss, img_pred)
        
print('Generation completed')

Generation completed


# 把cut_size1大小的图片合成一张整图

In [71]:
# if not os.path.exists(DATA_DIR + '/train960_normalize'):
#     os.makedirs(DATA_DIR + '/train960_normalize')

for year in years:
    img = np.ones((IM_ROWS, IM_COLS))
    ss = '{}/tiny_{}.tif'.format(DATA_DIR, str(year)[-2:])
    for i in xrange(int(IM_ROWS // cut_size1)):
        for j in xrange(int(IM_COLS // cut_size1)):
            ss1 = '{}/result_pred960/{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, cut_size1)
            img[i*cut_size1:(i+1)*cut_size1,j*cut_size1:(j+1)*cut_size1] = tiff.imread(ss1)

    if IM_ROWS%cut_size1 != 0:
        i += 1
        for k in xrange(int(IM_COLS // cut_size1)):
            ss1 = '{}/result_pred960/{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, k, cut_size1)
            img[i*cut_size1:IM_ROWS, k*cut_size1:(k+1)*cut_size1] = tiff.imread(ss1)[cut_size1-(IM_ROWS-i*cut_size1):, :]

    if IM_COLS%cut_size1 != 0:
        j += 1
        for k in xrange(int(IM_ROWS // cut_size1)):
            ss1 = '{}/result_pred960/{}_{}_{}_{}.tif'.format(DATA_DIR, year, k, j, cut_size1)
            img[k*cut_size1:(k+1)*cut_size1, j*cut_size1:IM_COLS] = tiff.imread(ss1)[:, cut_size1-(IM_COLS-j*cut_size1):]

    if IM_ROWS%cut_size1 != 0 and IM_COLS%cut_size1 != 0:
        ss1 = '{}/result_pred960/{}_{}_{}_{}.tif'.format(DATA_DIR, year, i, j, cut_size1)
        img[i*cut_size1:IM_ROWS, j*cut_size1:IM_COLS] = tiff.imread(ss1)[cut_size1-(IM_ROWS-i*cut_size1):, cut_size1-(IM_COLS-j*cut_size1):]
    tiff.imsave(ss, np.array(img*255, dtype=np.uint8))
        
print('Generation completed')

Generation completed
